<a href="https://colab.research.google.com/github/SamudralaAnuhya/Movie-recommender-systems/blob/main/Movie_recomender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 𝑛𝑢=443users, and 𝑛𝑚=4778movies.
# X(i)-- movies features (i)...genre
# W(j)---users (j)....
# y(i,j)--actual rating
# r(i,j)----if rating there then 1 else 0
#b(i,j)--all users

#Movie Recommender System (Collabirative filtering)
The data set is derived from the [movielens](https://grouplens.org/datasets/movielens/latest/) dataset.


Implementing from scratch and using tensor flow



<a name="2.3"></a>
### 4.1 Collaborative filtering cost function

The collaborative filtering cost function is given by
$$J({\mathbf{x}^{(0)},...,\mathbf{x}^{(n_m-1)},\mathbf{w}^{(0)},b^{(0)},...,\mathbf{w}^{(n_u-1)},b^{(n_u-1)}})= \left[ \frac{1}{2}\sum_{(i,j):r(i,j)=1}(\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)} - y^{(i,j)})^2 \right]
+ \underbrace{\left[
\frac{\lambda}{2}
\sum_{j=0}^{n_u-1}\sum_{k=0}^{n-1}(\mathbf{w}^{(j)}_k)^2
+ \frac{\lambda}{2}\sum_{i=0}^{n_m-1}\sum_{k=0}^{n-1}(\mathbf{x}_k^{(i)})^2
\right]}_{regularization}
\tag{1}$$
The first summation in (1) is "for all $i$, $j$ where $r(i,j)$ equals $1$" and could be written:

$$
= \left[ \frac{1}{2}\sum_{j=0}^{n_u-1} \sum_{i=0}^{n_m-1}r(i,j)*(\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)} - y^{(i,j)})^2 \right]
+\text{regularization}
$$

You should now write cofiCostFunc (collaborative filtering cost function) to return this cost.

In [2]:
import numpy as np
import pandas as pd
from numpy import loadtxt
import tensorflow as tf
from tensorflow import keras

In [16]:
def files_import_XWB():
  file = open('/content/drive/MyDrive/Move_recomendation/small_movies_X.csv' , 'rb')   #rb--read binary mode
  X = loadtxt(file,delimiter=",")

  file = open('/content/drive/MyDrive/Move_recomendation/small_movies_W.csv' , 'rb')
  W = loadtxt(file,delimiter=",")

  file = open('/content/drive/MyDrive/Move_recomendation/small_movies_b.csv' , 'rb')
  b = loadtxt(file,delimiter=",")

  b = b.reshape(1,-1)
  num_movies , num_features = X.shape
  num_users,_ = W.shape
  return(X, W, b, num_movies, num_features, num_users)



def files_import_ratingYR():   #Y--actual rating , r -- 0 if there is no rating or 1 if rating is there
  file = open('/content/drive/MyDrive/Move_recomendation/small_movies_Y.csv','rb')
  Y = loadtxt(file,delimiter= ",")

  file = open('/content/drive/MyDrive/Move_recomendation/small_movies_R.csv' ,'rb')
  R = loadtxt(file , delimiter=",")
  return(Y,R)




from scratch implementation

In [21]:
#load data
X, W, b, num_movies, num_features, num_users = files_import_XWB()
Y,R = files_import_ratingYR()

print("X" , X.shape)
print("W" , W.shape)
print("b" ,b.shape)
print("Y" , Y.shape  , "R" , R.shape)
print("num_movies" , num_movies)
print("num_users" , num_users)
print("num_features" , num_features)

X (4778, 10)
W (443, 10)
b (1, 443)
Y (4778, 443) R (4778, 443)
num_movies 4778
num_users 443
num_features 10


In [49]:
#filtering function is similar to gradient descent (wx+b)+regularization term
#here J= (r*(wx+b-y)**2)/2 + lamda/2(w**2+x**2)..cost function

def cofi_cost_func (X, W, b, Y, R, lambda_):    #collabirative cost function
    J = 0
    ### START CODE HERE ###
    for j in range(nu):
        w = W[j,:]
        b_j = b[0,j]
        for i in range(nm):
            x = X[i,:]
            y = Y[i,j]
            r = R[i,j]
            J += np.square(r * (np.dot(w,x) + b_j - y ) )
    J = J/2
    J += (lambda_/2) * (np.sum(np.square(W)) + np.sum(np.square(X)))
    return J

In [45]:
J = cofi_cost_func(X, W, b, Y, R, 1.6);
print(f"Cost: {J:0.2f}")   #we are getting big cost function need to do normalize the data

Cost: 308883.78


#using Tensorflow (vector implementaion)
we have to tun w-alphha d/dw...to reduce loss for that here we are using adam
1. implement cost function (vectiorized) for faster caliculation

2.find mean , norm

3.apply adams algorithm but before doung that we have to make as variable which values we are updating (means taking that value and passing again)..here for w,x,b

4.automatically dervative term will be generated and do calicuation by using GradientTape

5.pass the cost_value withn ynorm

6.pick values and again send to ptimizer do few iterations



In [46]:
def cofi_cost_func_v(X, W, b, Y, R, lambda_):
  j = (tf.linalg.matmul(X, tf.transpose(W)) + b - Y)*R
  J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(X**2) + tf.reduce_sum(W**2))
  return J

In [47]:
J = cofi_cost_func_v(X, W, b, Y, R, 1.6);
print(f"Cost: {J:0.2f}")

Cost: 308883.78


optimizer/normalization

In [54]:
Ymean =  (np.sum(Y * R , axis =1)/(np.sum( R , axis =1)+1e-12)).reshape(-1,1)   # doinf sum for only gicen rating /total rating
Ynorm = Y - np.multiply(Ymean, R)

#for training model find derivative terms of X,W,B VALUES
num_movies, num_users = Y.shape
num_features = 100

tf.random.set_seed(1234)
W = tf.Variable(tf.random.normal((num_users,  num_features),dtype=tf.float64),  name='W')
X = tf.Variable(tf.random.normal((num_movies, num_features),dtype=tf.float64),  name='X')
b = tf.Variable(tf.random.normal((1,          num_users),   dtype=tf.float64),  name='b')

optimizer = keras.optimizers.Adam(learning_rate=1e-1)

In [56]:
iterations = 200
lambda_ = 1
for iter in range(iterations):   #calicualte cost
  with tf.GradientTape() as tape :
    cost_value =cofi_cost_func_v(X, W, b, Ynorm, R, lambda_)
  grads = tape.gradient(cost_value,[X,W,b]) #pick gradient descents of each step
  optimizer.apply_gradients(zip(grads,[X,W,b])) # and send again descents to minimize the loss

  # Log periodically.
  if iter % 20 == 0:
       print(f"Training loss at iteration {iter}: {cost_value:0.1f}")




Training loss at iteration 0: 2287998.9
Training loss at iteration 20: 132726.4
Training loss at iteration 40: 49977.3
Training loss at iteration 60: 23645.4
Training loss at iteration 80: 13123.3
Training loss at iteration 100: 8205.2
Training loss at iteration 120: 5642.8
Training loss at iteration 140: 4209.9
Training loss at iteration 160: 3368.3
Training loss at iteration 180: 2854.8


In [62]:
def load_Movie_List_pd():
    """ returns df with and index of movies in the order they are in in the Y matrix """
    df = pd.read_csv('/content/drive/MyDrive/Move_recomendation/small_movie_list.csv', header=0, index_col=0,  delimiter=',', quotechar='"')
    mlist = df["title"].to_list()
    return(mlist, df)

In [63]:
movieList, movieList_df = load_Movie_List_pd()
filter=(movieList_df["number of ratings"] > 20)
movieList_df["pred"] = my_prediction
movieList_df = movieList_df.reindex(columns=["pred", "mean rating", "number of ratings", "title"])
movieList_df.loc[ix[:300]].loc[filter].sort_values("mean rating", ascending=False)

,pred,mean rating,number of ratings,title
2173,5.531714,4.057692,104,WALL·E (2008)
3618,5.637669,3.993151,73,Interstellar (2014)
